In [1]:
# importing some useful libraries
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import seaborn as sns  
import time 

from sklearn.metrics import classification_report
from sklearn.metrics import confusion_matrix
from nltk.tokenize import RegexpTokenizer  
from nltk.stem.snowball import SnowballStemmer
from sklearn.feature_extraction.text import CountVectorizer  
from sklearn.pipeline import make_pipeline

from sklearn.model_selection import train_test_split
from sklearn.linear_model import LogisticRegression


from PIL import Image
from wordcloud import WordCloud, STOPWORDS, ImageColorGenerator
import pickle 

# Wczytywanie danych

In [2]:
kaggle = pd.read_csv("../Data/URLs-mixed/kaggle_labeled.csv")
kaggle.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 420464 entries, 0 to 420463
Data columns (total 2 columns):
 #   Column  Non-Null Count   Dtype 
---  ------  --------------   ----- 
 0   url     420464 non-null  object
 1   label   420464 non-null  object
dtypes: object(2)
memory usage: 6.4+ MB


## Dane polska

In [3]:
gov = pd.read_csv("../Data/URLs-good/gov_poland.csv", skiprows=2, sep=';', usecols=[1], names=["url"])
gov.head(2)

,url
0,http://1bcz.wp.mil.pl
1,http://1bdm.wp.mil.pl/pl/28.html


In [4]:
gov["url"] = gov["url"].astype(str)
gov["label"] = "good"
gov.head(2)

,url,label
0,http://1bcz.wp.mil.pl,good
1,http://1bdm.wp.mil.pl/pl/28.html,good


## Dane cert

In [5]:
cert = pd.read_csv("https://hole.cert.pl/domains/domains.txt", names=["url"])
cert.head(2)

,url
0,008753331120.com
1,02-wiadomosci.com.pl


In [6]:
cert["url"] = cert["url"].astype(str)
cert["label"] = "bad"
cert.head(2)

,url,label
0,008753331120.com,bad
1,02-wiadomosci.com.pl,bad


In [7]:
df = pd.concat([kaggle,gov,cert])
df.head(2)

,url,label
0,diaryofagameaddict.com,bad
1,espdesign.com.au,bad


In [8]:
df["url"] = df.url.astype(str)

# Trenowanie modelu

In [9]:
def count_special_symbols(domain):
    counter = 0
    for char in domain:
        if char.isalpha() or char.isdigit():
            continue
        else:
            counter += 1
    return counter


def count_digits(domain):
    counter = 0
    for char in domain:
        if char.isdigit():
            counter += 1
    return counter

In [10]:
from urllib3.util import parse_url
import re

def parseurl(url):
    try:
        url = url.translate({'[': None, ']': None})
        url = parse_url(url)
        return str(url.host)
    except Exception as e:
        if "//" in url:
            url = url.split("//")[1]
        url = url[:url.find("/")]
        return str(url)
    
def parseurl2(url):
    try:
        url = url.translate({'[': "", ']': ""})
        h = re.search("https?:?//", url)
        if h is not None:
            url = url[:h.start()] + url[h.end():]
        return str(parse_url(url).host)
    except:
        return np.nan

In [11]:
df["url"] = df["url"].astype(str)
df["parsed_url"] = df.url.apply(parseurl2)

In [12]:
df.dropna(inplace=True)

In [13]:
good_domains = df[df["label"] == "good"]
bad = df[df["label"] == "bad"]

In [14]:
df.head(5)

,url,label,parsed_url
0,diaryofagameaddict.com,bad,diaryofagameaddict.com
1,espdesign.com.au,bad,espdesign.com.au
2,iamagameaddict.com,bad,iamagameaddict.com
3,kalantzis.net,bad,kalantzis.net
4,slightlyoffcenter.net,bad,slightlyoffcenter.net


In [15]:
data = pd.DataFrame(df.iloc[1:200,2])
data.head(5)

,parsed_url
1,espdesign.com.au
2,iamagameaddict.com
3,kalantzis.net
4,slightlyoffcenter.net
5,toddscarwash.com


In [16]:
data.columns = ["url"]

In [17]:
import Levenshtein

def get_best_lev_match(domains: pd.DataFrame):
    """
    Calculates Levenshtein distance of given string to all domains in top-100 polish database and returns best domain
    and it's score.

    Returns
    -------

    """

    known_domains = pd.DataFrame(good_domains['parsed_url'].str.split(".", n=1, expand=True)[0])
    known_domains.columns = ["url"]
    known_domains = known_domains.iloc[1:10000,:]

    return domains.apply(lambda y: 
        known_domains.apply(lambda x: Levenshtein.jaro_winkler(y['url'], x['url'], 0.25), axis=1).max(), axis=1)

In [18]:
from multiprocessing import  Pool
import random
def parallelize_dataframe(df__, func, n_cores=4):
    df_split = np.array_split(df__, n_cores)
    pool = Pool(n_cores)
    df_ = pd.concat(pool.map(func, df_split))
    pool.close()
    pool.join()
    return df_

In [ ]:
get_best_lev_match(data)

In [ ]:
train = parallelize_dataframe(data, get_best_lev_match,n_cores=16)

In [74]:
parse_url("pm137lodz.wikom.pl")

Url(scheme=None, auth=None, host='pm137lodz.wikom.pl', port=None, path=None, query=None, fragment=None)

In [95]:
re.search("https?:?\/\/", "pm137lodz.wikom.pl")